In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# YOLO

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.0/883.0 kB 12.8 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import requests
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import Point, box
from tqdm import tqdm
from ultralytics import YOLO
from io import BytesIO

# Load the trained YOLO model
checkpoint_path = '/content/drive/MyDrive/aerial_image_recognition/img/train/Tokyo/yolov8_tokyo_checkpoint.pt'
model = YOLO(checkpoint_path)
print("Model loaded from checkpoint successfully!")

# Load the 'ramki.shp' layer using Geopandas
ramki_shp_path = '/content/drive/MyDrive/aerial_image_recognition/gis/shp/ramki.shp'
ramki_gdf = gpd.read_file(ramki_shp_path)

# Select the specific area named 'srodmiescie' (adjust this filter for your case)
srodmiescie_gdf = ramki_gdf[ramki_gdf['name'] == 'srodmiescie']

# Get the bounding box of 'srodmiescie'
minx, miny, maxx, maxy = srodmiescie_gdf.total_bounds
print(f"Bounding box of srodmiescie: {minx}, {miny}, {maxx}, {maxy}")

# Define WMS parameters
wms_url = "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/HighResolution"
wms_params = {
    'service': 'WMS',
    'version': '1.3.0',
    'request': 'GetMap',
    'layers': 'ORTO',
    'styles': '',
    'crs': 'EPSG:3857',  # Match to your projected CRS (Web Mercator)
    'bbox': f'{minx},{miny},{maxx},{maxy}',  # Update for each tile
    'width': 1200,  # Set the image size matching your model
    'height': 1200,
    'format': 'image/tiff'
}

# Define an empty GeoDataFrame for storing car/truck detections
car_centroids_gdf = gpd.GeoDataFrame(columns=['geometry', 'class', 'confidence'])

# Loop through and fetch tiles from the WMS server
stride = 600  # Adjust stride for partial overlap
tile_size = 1200

# Split the area into smaller tiles based on stride and tile size
x_coords = list(range(int(minx), int(maxx), stride))
y_coords = list(range(int(miny), int(maxy), stride))

total_tiles = len(x_coords) * len(y_coords)

with tqdm(total=total_tiles, desc="Processing Tiles") as pbar:
    for x in x_coords:
        for y in y_coords:
            # Adjust bounding box for each tile
            tile_bbox = f'{x},{y},{x+tile_size},{y+tile_size}'
            wms_params['bbox'] = tile_bbox

            # Request the tile from WMS
            response = requests.get(wms_url, params=wms_params)

            if response.status_code == 200:
                # Load the image into OpenCV
                img_data = BytesIO(response.content)
                img = gdal.Open(img_data).ReadAsArray()
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                # Run YOLO inference on the tile
                results = model(img)

                # Extract bounding boxes and append to GeoDataFrame
                for box in results[0].boxes:
                    if box.cls in [0, 1] and box.conf > 0.4:  # Only consider cars/trucks
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        centroid_x = (x1 + x2) / 2 + x  # Adjust based on the tile's position
                        centroid_y = (y1 + y2) / 2 + y

                        # Append the result to the GeoDataFrame
                        car_centroids_gdf = car_centroids_gdf.append({
                            'geometry': Point(centroid_x, centroid_y),
                            'class': 'Car' if box.cls == 0 else 'Truck',
                            'confidence': box.conf.item()
                        }, ignore_index=True)

            pbar.update(1)

# Set CRS to match the WMS (EPSG:3857 for Web Mercator)
car_centroids_gdf.set_crs(epsg=3857, inplace=True)

# Save the results to a GeoJSON file
output_geojson_path = '/content/drive/MyDrive/aerial_image_recognition/gis/car_centroids_2.geojson'
car_centroids_gdf.to_file(output_geojson_path, driver='GeoJSON')

print(f"Car and truck centroids saved to {output_geojson_path}")


Model loaded from checkpoint successfully!
Bounding box of srodmiescie: 20.981389979899493, 52.22652066381057, 21.029888441054208, 52.25128290846448


Processing Tiles: 0it [00:00, ?it/s]

Car and truck centroids saved to /content/drive/MyDrive/aerial_image_recognition/gis/car_centroids_2.geojson


# ONNX

In [6]:
!pip install onnxruntime
!pip install owslib
!pip install geopandas
!pip install pyproj
!pip install tqdm
!pip install dnspython

In [13]:
import onnxruntime as ort
import numpy as np
import cv2
from owslib.wms import WebMapService
import geopandas as gpd
from shapely.geometry import Point, box
import io
from PIL import Image
import os
from google.colab import drive
from tqdm import tqdm
import time
import gc
import logging
import math
from pyproj import Transformer
from shapely.ops import transform
import pyproj

# Configuration settings
CONFIG = {
    'wms_url': "https://mapy.geoportal.gov.pl/wss/service/PZGIK/ORTO/WMS/HighResolution",
    'tile_size_meters': 51.2,      # 50m tiles
    'pixel_size': 0.08,          # 8cm/pixel
    'model_input_size': 640,     # Model input size
    'confidence_threshold': 0.3,  # Lower confidence threshold (was 0.5)
    'tile_overlap': 0.2,         # 20% overlap between tiles
    'duplicate_distance_threshold': 2e-5  # For removing duplicates from overlapping areas
}

def setup_logging(base_dir):
    """Minimal logging configuration - file only, no console output"""
    log_path = os.path.join(base_dir, 'detection_debug.log')

    # Suppress all logging to console
    logging.getLogger().handlers = []

    # File-only handler
    file_handler = logging.FileHandler(log_path, mode='w')
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

    # Root logger setup
    root_logger = logging.getLogger()
    root_logger.addHandler(file_handler)
    root_logger.setLevel(logging.INFO)

    # Suppress specific loggers that might be noisy
    logging.getLogger('owslib').setLevel(logging.WARNING)
    logging.getLogger('urllib3').setLevel(logging.WARNING)

    return log_path

def setup_environment():
    """Setup paths and mount Google Drive"""
    drive.mount('/content/drive', force_remount=True)
    base_dir = "/content/drive/Othercomputers/My_laptop/car_recognition"

    # Setup logging first
    log_path = setup_logging(base_dir)

    return {
        'model_path': os.path.join(base_dir, "models/car_aerial_detection_yolo7_ITCVD_deepness.onnx"),
        'frame_path': os.path.join(base_dir, "gis/shp/ramki/test_frame.shp"),
        'output_path': os.path.join(base_dir, 'gis/shp/car_detections_wms.geojson'),
        'tiles_path': os.path.join(base_dir, 'gis/shp/processing_tiles.geojson'),
        'log_path': log_path
    }

def meters_to_degrees(meters, latitude):
    """Convert meters to degrees at a given latitude"""
    earth_radius = 6378137
    degrees_longitude = meters / (earth_radius * math.cos(math.radians(latitude)) * 2 * math.pi / 360)
    degrees_latitude = meters / (earth_radius * 2 * math.pi / 360)
    return degrees_longitude, degrees_latitude

def get_tile_bboxes(bbox, tile_size_meters):
    """Generate overlapping tile bounding boxes"""
    minx, miny, maxx, maxy = bbox

    # Calculate tile size in degrees at the middle latitude
    mid_lat = (miny + maxy) / 2
    tile_size_lon, tile_size_lat = meters_to_degrees(tile_size_meters, mid_lat)

    # Calculate overlap in degrees
    overlap_lon = tile_size_lon * CONFIG['tile_overlap']
    overlap_lat = tile_size_lat * CONFIG['tile_overlap']

    # Calculate step size (tile size minus overlap)
    step_lon = tile_size_lon * (1 - CONFIG['tile_overlap'])
    step_lat = tile_size_lat * (1 - CONFIG['tile_overlap'])

    tile_bboxes = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            tile_bbox = (
                x,
                y,
                min(x + tile_size_lon, maxx),
                min(y + tile_size_lat, maxy)
            )
            tile_bboxes.append(tile_bbox)
            y += step_lat
        x += step_lon

    return tile_bboxes

def get_wms_image(wms, bbox):
    """Simplified WMS image retrieval without logging"""
    width_pixels = height_pixels = CONFIG['model_input_size']

    for attempt in range(3):
        try:
            img = wms.getmap(
                layers=['Raster'],
                srs='EPSG:4326',
                bbox=bbox,
                size=(width_pixels, height_pixels),
                format='image/png',
                transparent=True
            )

            return Image.open(io.BytesIO(img.read())).convert('RGB')

        except Exception as e:
            if attempt == 2:
                raise
            time.sleep(1)

def analyze_model(session):
    """Analyze ONNX model structure and outputs"""
    # Get model inputs
    inputs = session.get_inputs()
    outputs = session.get_outputs()

    logging.info("\nModel Analysis:")
    logging.info("Input details:")
    for input in inputs:
        logging.info(f"Name: {input.name}")
        logging.info(f"Shape: {input.shape}")
        logging.info(f"Type: {input.type}")

    logging.info("\nOutput details:")
    for output in outputs:
        logging.info(f"Name: {output.name}")
        logging.info(f"Shape: {output.shape}")
        logging.info(f"Type: {output.type}")

    # The number of classes can be inferred from output shape
    # YOLO output shape is typically [batch, num_boxes, num_classes + 5]
    num_classes = outputs[0].shape[2] - 5  # subtract 5 for box coords + confidence
    logging.info(f"\nNumber of classes: {num_classes}")

    return num_classes

def process_detections(outputs, tile_bbox, num_classes):
    """Process model outputs with class information"""
    detections = []
    boxes = outputs[0][0]

    for box in boxes:
        # Get confidence and class scores
        confidence = box[4]
        if confidence > CONFIG['confidence_threshold']:
            # YOLO format: center_x, center_y, width, height, confidence, [class_scores]
            center_x, center_y, width, height = box[:4]

            # Get class probabilities and best class
            class_scores = box[5:5+num_classes]
            class_id = np.argmax(class_scores)
            class_score = class_scores[class_id]

            # Calculate geographic coordinates as before
            norm_x = center_x / CONFIG['model_input_size']
            norm_y = center_y / CONFIG['model_input_size']

            longitude = tile_bbox[0] + (norm_x * (tile_bbox[2] - tile_bbox[0]))
            latitude = tile_bbox[3] - (norm_y * (tile_bbox[3] - tile_bbox[1]))

            detections.append({
                'geometry': Point(longitude, latitude),
                'confidence': float(confidence),
                'class_id': int(class_id),
                'class_score': float(class_score),
                'bbox': [float(x) for x in box[:4]],
                'bbox_width': float(width),
                'bbox_height': float(height)
            })

    return detections

def improve_detection(img):
    """Simplified image preprocessing with most effective enhancements"""
    # Only keep the most effective enhancements
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    l_enhanced = clahe.apply(l)
    enhanced_lab = cv2.merge((l_enhanced, a, b))
    enhanced = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2RGB)

    return [img, enhanced]

def process_multiscale(wms, session, tile_bbox, num_classes, scales=[1.0]):
    """Simplified multiscale processing with fewer scales"""
    all_detections = []

    for scale in scales:
        try:
            original_size = CONFIG['tile_size_meters']
            scaled_size = original_size * scale
            center_lon = (tile_bbox[0] + tile_bbox[2]) / 2
            center_lat = (tile_bbox[1] + tile_bbox[3]) / 2
            size_lon, size_lat = meters_to_degrees(scaled_size, center_lat)

            scaled_bbox = (
                center_lon - size_lon/2,
                center_lat - size_lat/2,
                center_lon + size_lon/2,
                center_lat + size_lat/2
            )

            img = get_wms_image(wms, scaled_bbox)
            if img is None:
                continue

            img_array = np.array(img)
            enhanced_images = improve_detection(img_array)

            for enhanced in enhanced_images:
                input_tensor = cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR)
                input_tensor = np.transpose(input_tensor, (2, 0, 1))
                input_tensor = np.expand_dims(input_tensor, axis=0).astype(np.float32) / 255.0

                outputs = session.run(None, {session.get_inputs()[0].name: input_tensor})
                detections = process_detections(outputs, scaled_bbox, num_classes)

                for det in detections:
                    det['detection_scale'] = scale
                    all_detections.append(det)

        except Exception as e:
            logging.error(f"Error processing tile at scale {scale}: {str(e)}")
            continue

    return all_detections

def remove_duplicates(detections_gdf):
    """Duplicate removal with 2m threshold and progress bar"""
    if len(detections_gdf) == 0:
        return detections_gdf

    # Create UTM transformer for accurate distances
    proj_utm = pyproj.CRS('EPSG:32634')  # UTM zone 34N
    proj_wgs = pyproj.CRS('EPSG:4326')
    project = pyproj.Transformer.from_crs(proj_wgs, proj_utm, always_xy=True).transform

    # Convert to UTM
    detections_utm = detections_gdf.copy()
    detections_utm.geometry = detections_utm.geometry.apply(lambda geom: transform(project, geom))

    # Sort by confidence
    detections_utm = detections_utm.sort_values('confidence', ascending=False)

    kept_indices = []
    used_indices = set()
    distance_threshold = 2.0  # 2 meters

    # Add progress bar
    with tqdm(total=len(detections_utm), desc="Removing duplicates", leave=False) as pbar:
        for idx in detections_utm.index:
            if idx in used_indices:
                pbar.update(1)
                continue

            kept_indices.append(idx)
            point = detections_utm.loc[idx, 'geometry']

            # Find nearby points
            for other_idx in detections_utm.index:
                if other_idx != idx and other_idx not in used_indices:
                    other_point = detections_utm.loc[other_idx, 'geometry']
                    distance = point.distance(other_point)

                    if distance < distance_threshold:
                        used_indices.add(other_idx)

            pbar.update(1)

    # Convert back to WGS84
    project_back = pyproj.Transformer.from_crs(proj_utm, proj_wgs, always_xy=True).transform
    filtered_gdf = detections_gdf.loc[kept_indices].copy()
    filtered_gdf.geometry = filtered_gdf.geometry.apply(lambda geom: transform(project_back, transform(project, geom)))

    return filtered_gdf

def main():
    """Main execution with minimal console output"""
    try:
        paths = setup_environment()
        start_time = time.time()

        # Initialize components silently
        wms = WebMapService(CONFIG['wms_url'], version='1.3.0')
        session = ort.InferenceSession(paths['model_path'])
        num_classes = analyze_model(session)

        # Load and process frame
        frame_gdf = gpd.read_file(paths['frame_path'])
        if frame_gdf.crs.to_epsg() != 4326:
            frame_gdf = frame_gdf.to_crs(epsg=4326)

        bbox = frame_gdf.total_bounds
        tile_bboxes = get_tile_bboxes(bbox, CONFIG['tile_size_meters'])

        # Process tiles with clean progress bar
        all_detections = []
        with tqdm(total=len(tile_bboxes), desc="Processing tiles", unit="tile") as pbar:
            for tile_bbox in tile_bboxes:
                try:
                    detections = process_multiscale(wms, session, tile_bbox, num_classes, scales=[0.8, 1.0, 1.2])
                    if detections:
                        all_detections.extend(detections)
                except Exception as e:
                    # Silently log error to file and continue
                    logging.error(f"Tile processing error: {str(e)}")
                finally:
                    pbar.update(1)
                    gc.collect()

        # Create and process detections DataFrame
        if not all_detections:
            detections_gdf = gpd.GeoDataFrame(
                columns=['geometry', 'confidence', 'class_id', 'class_score', 'bbox'],
                crs="EPSG:4326"
            )
        else:
            detections_gdf = gpd.GeoDataFrame(all_detections, crs="EPSG:4326")

            # Show progress for duplicate removal
            with tqdm(total=1, desc="Removing duplicates", leave=True) as pbar:
                detections_gdf = remove_duplicates(detections_gdf)
                pbar.update(1)

        # Save results
        detections_gdf.to_file(paths['output_path'], driver='GeoJSON')

        # Final summary - just two lines
        minutes = (time.time() - start_time) / 60
        print(f"\nCompleted in {minutes:.1f} min. Found {len(detections_gdf)} objects.")

        return detections_gdf

    except Exception as e:
        print(f"\nError occurred. Check detection_debug.log for details.")
        logging.error(f"Processing failed: {str(e)}")
        return gpd.GeoDataFrame(
            columns=['geometry', 'confidence', 'class_id', 'class_score', 'bbox'],
            crs="EPSG:4326"
        )

if __name__ == "__main__":
    detections_gdf = main()

    # Print summary
    print(f"\nProcessing Summary:")
    print(f"Total tiles processed: {len(tiles_gdf)}")
    print(f"Total detections: {len(detections_gdf)}")
    if len(detections_gdf) > 0:
        print(f"Average confidence: {detections_gdf['confidence'].mean():.3f}")

Mounted at /content/drive


Removing duplicates: 100%|██████████| 1/1 [00:54<00:00, 54.79s/it]



Completed in 9.7 min. Found 301 objects.

Processing Summary:
Total tiles processed: 0
Total detections: 301
Average confidence: 0.774
